Overview of MLflow

In this notebook you will build a 

In [13]:
import mlflow
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from pathlib import Path
import os

In [23]:
os.chdir("/workspaces/mlflow-training")

# Ingest data

In [24]:
red_df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=";")
red_df["is_red"] = 1
white_df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv", sep=";")
white_df["is_red"] = 0

df = pd.concat([red_df, white_df])
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,is_red
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,0
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,0
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,0
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,0


# Split data

In [25]:
y = df[["quality"]]
X = df.drop("quality", axis=1, inplace=False)

X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42)

# Transform data

In [26]:
# NA

# Train model

In [30]:
def setup_mlflow(experiment_name, mlflow_tracking_uri):
    # Get a list of all existing experiments
    experiments = mlflow.search_experiments()
    
    # experiments = mlflow.list_experiments()
    experiment_names = [ex.name for ex in experiments]
    # Create experiment if it does not exist
    if experiment_name not in experiment_names:
        mlflow.create_experiment(
            experiment_name,
            artifact_location=Path.cwd().parent.joinpath("metadata/mlflow/mlartifacts").as_uri()
        )
    mlflow.set_experiment(experiment_name)
    mlflow.set_tracking_uri(mlflow_tracking_uri)

# Usage
setup_mlflow(
    experiment_name="wine_score_notebook",
    mlflow_tracking_uri="sqlite:////workspaces/mlflow-training/src/metadata/mlflow/mlruns.db"
)

In [31]:
model = LinearRegression()

mlflow.autolog(log_input_examples=True)
with mlflow.start_run():
    model.fit(X_train, y_train)
    
mlflow.autolog(disable=False)

2023/05/11 20:13:13 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2023/05/11 20:13:14 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/nonroot/.local/lib/python3.10/site-packages/mlflow/models/signature.py:137: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2023/05/11 20:13:16 INFO mlflow.tracking.fluent: Au

# 